In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import make_classification

In [2]:
torch.manual_seed(42)

In [3]:
if torch.cuda.is_available():
    print("GPU available.")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
elif torch.mps.is_available():
    print("MPS available.")
    print(f"Using MPS: {torch.mps.device_count()}")
    device = torch.device("mps")
else:
    print("Using CPU.")
    device = torch.device("cpu")

device = torch.device("cpu")

torch.set_default_device(device=device)

MPS available.
Using MPS: 1


In [ ]:
df = pd.read_csv('fmnist_small.csv')
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [48]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [49]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [50]:
x_train_tensor = torch.from_numpy(x_train).to(dtype=torch.float32, device=device)
x_test_tensor = torch.from_numpy(x_test).to(dtype=torch.float32, device=device)
y_train_tensor = torch.from_numpy(y_train).to(dtype=torch.float32, device=device)
y_test_tensor = torch.from_numpy(y_test).to(dtype=torch.float32, device=device)

In [51]:
class CancerDataset(Dataset):

    def __init__(self, feartures, labels):
        self.features = feartures
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [53]:
train_dataset = CancerDataset(x_train_tensor, y_train_tensor)
test_dataset = CustomDataset(x_test_tensor, y_test_tensor)

In [62]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [56]:
class NeuralNetwork(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=3, device=device),
            nn.ReLU(),
            nn.Linear(in_features=3, out_features=1, device=device),
            nn.Sigmoid(),
        )
    
    def forward(self, features):
        out = self.network(features)
        return out

In [ ]:
lr = 0.1
epochs = 1000

In [58]:
model = NeuralNetwork(x_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=lr)

loss_function = nn.BCELoss()

In [63]:
for epoch in range(epochs):
    for batch_features, batch_labels in train_loader:
        y_pred = model(batch_features)
        loss = loss_function(y_pred, batch_labels.view(-1, 1))
        optimizer.zero_grad()
        optimizer.step()
    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.6178330183029175
Epoch: 2, Loss: 0.5962808728218079
Epoch: 3, Loss: 0.6785789132118225
Epoch: 4, Loss: 0.6505520939826965
Epoch: 5, Loss: 0.6888703107833862
Epoch: 6, Loss: 0.7066325545310974
Epoch: 7, Loss: 0.6415525078773499
Epoch: 8, Loss: 0.639633059501648
Epoch: 9, Loss: 0.7033472061157227
Epoch: 10, Loss: 0.7057944536209106
Epoch: 11, Loss: 0.6513578295707703
Epoch: 12, Loss: 0.6787814497947693
Epoch: 13, Loss: 0.645814061164856
Epoch: 14, Loss: 0.7108530402183533
Epoch: 15, Loss: 0.6646132469177246
Epoch: 16, Loss: 0.6916759610176086
Epoch: 17, Loss: 0.630558431148529
Epoch: 18, Loss: 0.614026665687561
Epoch: 19, Loss: 0.6813562512397766
Epoch: 20, Loss: 0.6561017632484436
Epoch: 21, Loss: 0.7056004405021667
Epoch: 22, Loss: 0.6673980355262756
Epoch: 23, Loss: 0.6674953103065491
Epoch: 24, Loss: 0.6532295942306519
Epoch: 25, Loss: 0.6856598258018494
Epoch: 26, Loss: 0.6778575778007507
Epoch: 27, Loss: 0.6821116209030151
Epoch: 28, Loss: 0.6642427444458008
Epoch

In [65]:
model.eval()
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.9).float()

        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy: .4f}')

Accuracy:  0.5747
